In [ ]:
!apt install swig cmake

In [ ]:
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

In [21]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
import gymnasium as gym

env = gym.make("MountainCar-v0")

observation, info = env.reset()

for _ in range(20):
  action = env.action_space.sample()
  print(f"Action taken: {action}")

  observation, reward, terminated, truncated, info = env.step(action)

  if terminated or truncated:
    print("Environment is reset")
    observation, info = env.reset()

env.close()

Action taken: 2
Action taken: 2
Action taken: 1
Action taken: 0
Action taken: 0
Action taken: 0
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 2
Action taken: 0
Action taken: 1
Action taken: 2
Action taken: 1
Action taken: 1
Action taken: 1
Action taken: 0


In [6]:
env = make_vec_env('MountainCar-v0')

In [30]:
model = DQN(
      policy="MlpPolicy",
      env=env,
      learning_rate=1e-3,
      buffer_size=50000,
      exploration_fraction=0.1,
      exploration_final_eps=0.1,
  )

In [31]:
model.learn(total_timesteps=500000)

model_name = 'dqn-MountainCar-v0'
model.save(model_name)

In [32]:
eval_env = Monitor(gym.make("MountainCar-v0"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

Mean reward: -163.90 +/- 36.20


In [10]:
notebook_login()
!git config --global credential.helper store

In [33]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

## TODO: Define a repo_id
## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
repo_id = 'kowalsky/MountainCar-v0'

# TODO: Define the name of the environment
env_id = 'MountainCar-v0'

# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])


# TODO: Define the model architecture we used
model_architecture = "PPO"

## TODO: Define the commit message
commit_message = "Added trained MountainCar-v0"

# method save, evaluate, generate a model card and record a replay video of your agent before pushing the repo to the hub
package_to_hub(model=model, # Our trained model
               model_name=model_name, # The name of our trained model
               model_architecture=model_architecture, # The model architecture we used: in our case PPO
               env_id=env_id, # Name of the environment
               eval_env=eval_env, # Evaluation Environment
               repo_id=repo_id, # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
               commit_message=commit_message)

ℹ This function will save, evaluate, generate a video of your agent,
create a model card and push everything to the hub. It might take up to 1min.
This is a work in progress: if you encounter a bug, please open an issue.
Saving video to /tmp/tmpro098g3d/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmpro098g3d/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmpro098g3d/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmpro098g3d/-step-0-to-step-1000.mp4
ℹ Pushing repo kowalsky/MountainCar-v0 to the Hugging Face Hub


policy.optimizer.pth:   0%|          | 0.00/41.3k [00:00<?, ?B/s]

policy.pth:   0%|          | 0.00/40.4k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

dqn-MountainCar-v0.zip:   0%|          | 0.00/97.3k [00:00<?, ?B/s]

ℹ Your model is pushed to the Hub. You can view your model here:
https://huggingface.co/kowalsky/MountainCar-v0/tree/main/


'https://huggingface.co/kowalsky/MountainCar-v0/tree/main/'